In [143]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, confusion_matrix, classification_report
#from sklearn.metrics import ConfusionMatrixDisplay
#import seaborn as sns
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, LeakyReLU
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.optimizers import Adam

stop_words = stopwords.words('english')

In [11]:
data = pd.read_csv('prepared_text_data_sugar.csv')
data

,sugar_class,text
0,5,"Cookie Dough Blizzard Cake, 10 in Cookie Dou..."
1,5,"Reeses Peanut Butter Cups Blizzard Cake, 10 ..."
2,5,"Chocolate Xtreme Blizzard Cake, 10 in Chocol..."
3,5,"Oreo Blizzard Cake, 10 in Oreo Blizzard Cake..."
4,5,"DQ Round Cake, 10 in DQ Round Cake, 10 in DQ..."
...,...,...
52926,1,"6 Nuggets 6 Nuggets 6 Nuggets, Tenders Entrees"
52927,1,"Breast, Bonafide Spicy Chicken Breast, Bonafi..."
52928,1,"Thigh, Bonafide Spicy Chicken Thigh, Bonafide..."
52929,1,"Leg, Bonafide Spicy Chicken Leg, Bonafide Spi..."


In [10]:
class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, data, y=None):
        return self

    def transform(self, data, y=None):
        preprocessed_data = [self.stem_doc(doc) for doc in data]
        return preprocessed_data

    def stem_doc(self, doc):
        stemmer = SnowballStemmer('english')
        lower_doc = [token.lower() for token in word_tokenize(doc) if token.isalpha()]
        filtered_doc = [token for token in lower_doc if token not in stop_words]
        stemmed_doc = [stemmer.stem(token) for token in filtered_doc]
        return " ".join(stemmed_doc)

In [129]:
X = data['text']
y = data['sugar_class']

#One-Hot_Encoding target variable, train/test split
ohe = OneHotEncoder(drop='first', sparse=False)
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, y, test_size = 0.2, random_state = 200)
y_train_raw = y_train_raw.values.reshape(-1, 1)
y_test_raw = y_test_raw.values.reshape(-1, 1)
y_train = ohe.fit_transform(y_train_raw - 1)
y_test = ohe.transform(y_test_raw - 1)

#Pre-processing and vectorizing text
processor = Preprocessor()
X_train_transformed = processor.fit_transform(X_train_raw)
X_test_transformed = processor.transform(X_test_raw)
vector_pipe = Pipeline([('tfidf', TfidfVectorizer())])
X_train_vector = vector_pipe.fit_transform(X_train_transformed)
X_test_vector = vector_pipe.transform(X_test_transformed)

#Returning independent variables to pd.Dataframe
X_train = pd.DataFrame(X_train_vector.toarray(), columns = vector_pipe['tfidf'].get_feature_names())
X_test = pd.DataFrame(X_test_vector.toarray(), columns = vector_pipe['tfidf'].get_feature_names())

In [157]:
#Ensuring X_train and X_test are proper data types
X_train_array = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
X_test_array = X_test.values if isinstance(X_test, pd.DataFrame) else X_test

#
y_train_reshaped = np.argmax(y_train_encoded, axis=2)

def f1_metric(y_true, y_pred):
    y_pred = tf.round(y_pred)
    true_positives = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32), axis=0)
    predicted_positives = tf.reduce_sum(tf.cast(y_pred, tf.float32), axis=0)
    actual_positives = tf.reduce_sum(tf.cast(y_true, tf.float32), axis=0)
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (actual_positives + tf.keras.backend.epsilon())
    
    f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
    return tf.reduce_mean(f1)

#
trainCallback = EarlyStopping(monitor='loss', min_delta = 1e-6, patience = 5)

reg = l2(0.0001)
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

In [158]:
model = Sequential()
model.add(Dense(1800, activation = LeakyReLU(), input_shape = (X_train_array.shape[1],), kernel_regularizer = reg))
model.add(Dense(4, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = [f1_metric])
model.fit(X_train_array, y_train_reshaped, epochs = 15, callbacks=[trainCallback], batch_size= 128, validation_split = 0.2)

Epoch 1/15


OperatorNotAllowedInGraphError: in user code:

    C:\Users\normc\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-157-7c95bd000fae>:10 f1  *
        f1 = f1_score(y_true, y_pred, average = 'weighted')
    C:\Users\normc\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:938 inner_f  *
        return f(**kwargs)
    C:\Users\normc\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1043 f1_score  *
        return fbeta_score(y_true, y_pred, beta=1, labels=labels,
    C:\Users\normc\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:938 inner_f  *
        return f(**kwargs)
    C:\Users\normc\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1166 fbeta_score  *
        _, _, f, _ = precision_recall_fscore_support(y_true, y_pred,
    C:\Users\normc\anaconda3\envs\learn-env\l

In [114]:
X_test.shape[1]

4005